In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import cv2

def hide_message_in_image(image_path, message, output_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        messagebox.showerror("Error", "Only RGB mode images are supported.")
        return

    pixels = img.load()
    width, height = img.size

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given image.")
        return

    index = 0
    for y in range(height):
        for x in range(width):
            if index < message_length:
                r, g, b = pixels[x, y]
                # Hide message in LSB of each channel
                r = (r & 0xFE) | int(binary_message[index])
                index += 1
                if index < message_length:
                    g = (g & 0xFE) | int(binary_message[index])
                    index += 1
                if index < message_length:
                    b = (b & 0xFE) | int(binary_message[index])
                    index += 1
                pixels[x, y] = (r, g, b)
            else:
                break
        else:
            continue
        break

    img.save(output_path)
    messagebox.showinfo("Success", "Message hidden successfully! Image saved as {}".format(output_path))

def decode_message_in_image(image_path):
    img = Image.open(image_path)
    pixels = img.load()
    width, height = img.size

    binary_message = ""
    for y in range(height):
        for x in range(width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    codec = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, codec, fps, (width, height))

    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)

    if message_length * 3 > width * height:
        messagebox.showerror("Error", "Message is too long for the given video.")
        return

    index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(height):
            for x in range(width):
                if index < message_length:
                    b, g, r = frame[y, x]
                    # Hide message in LSB of each channel
                    r = (r & 0xFE) | int(binary_message[index])
                    index += 1
                    if index < message_length:
                        g = (g & 0xFE) | int(binary_message[index])
                        index += 1
                    if index < message_length:
                        b = (b & 0xFE) | int(binary_message[index])
                        index += 1
                    frame[y, x] = [b, g, r]
                else:
                    break

        out.write(frame)

    cap.release()
    out.release()
    messagebox.showinfo("Success", "Message hidden successfully! Video saved as {}".format(output_path))

def decode_message_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    binary_message = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for y in range(frame.shape[0]):
            for x in range(frame.shape[1]):
                b, g, r = frame[y, x]
                binary_message += str(r & 1)
                binary_message += str(g & 1)
                binary_message += str(b & 1)

    message = ""
    for i in range(0, len(binary_message), 8):
        byte = binary_message[i:i+8]
        char = chr(int(byte, 2))
        if char == '\x00':
            break
        message += char

    return message.strip('\x00')

class SteganographyApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Media Steganography App")

        # Image encoding section
        self.image_encode_frame = tk.Frame(self.master)
        self.image_encode_frame.pack()

        self.image_label = tk.Label(self.image_encode_frame, text="Select an image:")
        self.image_label.pack()

        self.image_path_entry = tk.Entry(self.image_encode_frame, state='disabled', width=50)
        self.image_path_entry.pack(side='left', padx=5)

        self.image_browse_button = tk.Button(self.image_encode_frame, text="Browse", command=self.browse_image)
        self.image_browse_button.pack(side='left', padx=5)

        self.image_message_label = tk.Label(self.image_encode_frame, text="Enter a message:")
        self.image_message_label.pack()

        self.image_message_entry = tk.Entry(self.image_encode_frame, width=50)
        self.image_message_entry.pack(padx=5)

        self.image_save_label = tk.Label(self.image_encode_frame, text="Choose output filename:")
        self.image_save_label.pack()

        self.image_save_entry = tk.Entry(self.image_encode_frame, width=50)
        self.image_save_entry.pack(padx=5)

        self.image_hide_button = tk.Button(self.image_encode_frame, text="Hide Message in Image", command=self.hide_message_in_image)
        self.image_hide_button.pack(pady=10)

        # Image decoding section
        self.image_decode_frame = tk.Frame(self.master)
        self.image_decode_frame.pack()

        self.image_decode_label = tk.Label(self.image_decode_frame, text="Decode Hidden Message from Image:")
        self.image_decode_label.pack()

        self.image_decode_path_entry = tk.Entry(self.image_decode_frame, state='disabled', width=50)
        self.image_decode_path_entry.pack(side='left', padx=5)

        self.image_decode_browse_button = tk.Button(self.image_decode_frame, text="Browse", command=self.browse_encoded_image)
        self.image_decode_browse_button.pack(side='left', padx=5)

        self.image_decode_button = tk.Button(self.image_decode_frame, text="Decode Image", command=self.decode_image)
        self.image_decode_button.pack(pady=10)

        # Video encoding section
        self.video_encode_frame = tk.Frame(self.master)
        self.video_encode_frame.pack()

        self.video_label = tk.Label(self.video_encode_frame, text="Select a video:")
        self.video_label.pack()

        self.video_path_entry = tk.Entry(self.video_encode_frame, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.video_browse_button = tk.Button(self.video_encode_frame, text="Browse", command=self.browse_video)
        self.video_browse_button.pack(side='left', padx=5)

        self.video_message_label = tk.Label(self.video_encode_frame, text="Enter a message:")
        self.video_message_label.pack()

        self.video_message_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_message_entry.pack(padx=5)

        self.video_save_label = tk.Label(self.video_encode_frame, text="Choose output filename:")
        self.video_save_label.pack()

        self.video_save_entry = tk.Entry(self.video_encode_frame, width=50)
        self.video_save_entry.pack(padx=5)

        self.video_hide_button = tk.Button(self.video_encode_frame, text="Hide Message in Video", command=self.hide_message_in_video)
        self.video_hide_button.pack(pady=10)

        # Video decoding section
        self.video_decode_frame = tk.Frame(self.master)
        self.video_decode_frame.pack()

        self.video_decode_label = tk.Label(self.video_decode_frame, text="Decode Hidden Message from Video:")
        self.video_decode_label.pack()

        self.video_decode_path_entry = tk.Entry(self.video_decode_frame, state='disabled', width=50)
        self.video_decode_path_entry.pack(side='left', padx=5)

        self.video_decode_browse_button = tk.Button(self.video_decode_frame, text="Browse", command=self.browse_encoded_video)
        self.video_decode_browse_button.pack(side='left', padx=5)

        self.video_decode_button = tk.Button(self.video_decode_frame, text="Decode Video", command=self.decode_video)
        self.video_decode_button.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_path_entry.config(state='normal')
        self.image_path_entry.delete(0, tk.END)
        self.image_path_entry.insert(0, file_path)
        self.image_path_entry.config(state='disabled')

    def hide_message_in_image(self):
        image_path = self.image_path_entry.get()
        message = self.image_message_entry.get()
        save_path = self.image_save_entry.get()

        if image_path and message and save_path:
            output_path = save_path + ".png"
            hide_message_in_image(image_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select an image, enter a message, and choose an output filename.")

    def browse_encoded_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        self.image_decode_path_entry.config(state='normal')
        self.image_decode_path_entry.delete(0, tk.END)
        self.image_decode_path_entry.insert(0, file_path)
        self.image_decode_path_entry.config(state='disabled')

    def decode_image(self):
        image_path = self.image_decode_path_entry.get()
        if image_path:
            message = decode_message_in_image(image_path)
            messagebox.showinfo("Decoded Message from Image", message)
        else:
            messagebox.showerror("Error", "Please select an encoded image to decode.")

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def hide_message_in_video(self):
        video_path = self.video_path_entry.get()
        message = self.video_message_entry.get()
        save_path = self.video_save_entry.get()

        if video_path and message and save_path:
            output_path = save_path + ".mp4"
            hide_message_in_video(video_path, message, output_path)
        else:
            messagebox.showerror("Error", "Please select a video, enter a message, and choose an output filename.")

    def browse_encoded_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        self.video_decode_path_entry.config(state='normal')
        self.video_decode_path_entry.delete(0, tk.END)
        self.video_decode_path_entry.insert(0, file_path)
        self.video_decode_path_entry.config(state='disabled')

    def decode_video(self):
        video_path = self.video_decode_path_entry.get()
        if video_path:
            message = decode_message_in_video(video_path)
            messagebox.showinfo("Decoded Message from Video", message)
        else:
            messagebox.showerror("Error", "Please select an encoded video to decode.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyApp(root)
    root.mainloop()


In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox

def decode_message(video_path):
    cap = cv2.VideoCapture(video_path)

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    # Extract the hidden message from the least significant bit of each pixel
    hidden_message = ""
    for frame in frames:
        for row in frame:
            for pixel in row:
                for color in pixel:
                    hidden_message += str(color & 1)

    # Convert binary message to text
    message = ""
    for i in range(0, len(hidden_message), 8):
        byte = hidden_message[i:i+8]
        message += chr(int(byte, 2))

    return message

class SteganographyDecoderApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Video Steganography Decoder")

        self.video_path_label = tk.Label(self.master, text="Select an encoded video:")
        self.video_path_label.pack()

        self.video_path_entry = tk.Entry(self.master, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.browse_button = tk.Button(self.master, text="Browse", command=self.browse_video)
        self.browse_button.pack(side='left', padx=5)

        self.decode_button = tk.Button(self.master, text="Decode Message", command=self.decode_message)
        self.decode_button.pack(pady=10)

        self.message_label = tk.Label(self.master, text="Decoded Message:")
        self.message_label.pack()

        self.message_display = tk.Text(self.master, width=50, height=5)
        self.message_display.pack()

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mkv")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def decode_message(self):
        video_path = self.video_path_entry.get()

        if video_path:
            try:
                message = decode_message(video_path)
                self.message_display.delete('1.0', tk.END)
                self.message_display.insert(tk.END, message)
                messagebox.showinfo("Decoded Message", "Message decoded successfully!")
            except Exception as e:
                messagebox.showerror("Error", "Failed to decode message. Ensure that the selected file contains a hidden message.")
        else:
            messagebox.showerror("Error", "Please select an encoded video.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyDecoderApp(root)
    root.mainloop()


In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox

def encode_message(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    
    # Get dimensions of the first frame
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video frames.")
        return

    height, width, _ = frame.shape
    target_size = (width, height)

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, target_size)
        frames.append(resized_frame)

    cap.release()

    # Convert the message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)

    frame_index = 0
    for char in binary_message:
        # Embed the message in the least significant bit of each pixel
        for i in range(3):  # Iterate over RGB channels
            frames[frame_index][:, :, i] = frames[frame_index][:, :, i].astype(np.uint8)  # Ensure dtype is uint8
            frames[frame_index][:, :, i] &= ~np.uint8(1)  # Clear the least significant bit
            frames[frame_index][:, :, i] |= np.uint8(int(char, 2))  # Set the least significant bit with the message bit

            frame_index += 1

            if frame_index >= len(frames):
                break

    # Save the video with the hidden message using the same codec and container format as the input video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = 30  # Set a default FPS value
    out = cv2.VideoWriter(output_path, fourcc, fps, target_size)
    for frame in frames:
        out.write(frame)
    out.release()

    return output_path

def decode_message(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Failed to open video file.")
        return

    # Get dimensions of the first frame
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read video frames.")
        return

    height, width, _ = frame.shape
    target_size = (width, height)

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, target_size)
        frames.append(resized_frame)

    cap.release()

    # Extract the hidden message from the least significant bit of each pixel
    hidden_message = ""
    for frame in frames:
        for row in frame:
            for pixel in row:
                for color in pixel:
                    hidden_message += str(color & 1)

    # Convert binary message to text
    message = ""
    for i in range(0, len(hidden_message), 8):
        byte = hidden_message[i:i+8]
        message += chr(int(byte, 2))

    return message

class SteganographyDecoderApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Video Steganography Decoder")

        self.video_path_label = tk.Label(self.master, text="Select an encoded video:")
        self.video_path_label.pack()

        self.video_path_entry = tk.Entry(self.master, state='disabled', width=50)
        self.video_path_entry.pack(side='left', padx=5)

        self.browse_button = tk.Button(self.master, text="Browse", command=self.browse_video)
        self.browse_button.pack(side='left', padx=5)

        self.decode_button = tk.Button(self.master, text="Decode Message", command=self.decode_message)
        self.decode_button.pack(pady=10)

        self.message_label = tk.Label(self.master, text="Decoded Message:")
        self.message_label.pack()

        self.message_display = tk.Text(self.master, width=50, height=5)
        self.message_display.pack()

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mkv")])
        self.video_path_entry.config(state='normal')
        self.video_path_entry.delete(0, tk.END)
        self.video_path_entry.insert(0, file_path)
        self.video_path_entry.config(state='disabled')

    def decode_message(self):
        video_path = self.video_path_entry.get()

        if video_path:
            try:
                message = decode_message(video_path)
                self.message_display.delete('1.0', tk.END)
                self.message_display.insert(tk.END, message)
                messagebox.showinfo("Decoded Message", "Message decoded successfully!")
            except Exception as e:
                messagebox.showerror("Error", "Failed to decode message. Ensure that the selected file contains a hidden message.")
        else:
            messagebox.showerror("Error", "Please select an encoded video.")

if __name__ == '__main__':
    root = tk.Tk()
    app = SteganographyDecoderApp(root)
    root.mainloop()


In [1]:
import cv2
import numpy as np

# Function to embed message in a video frame
def embed_message(frame, message):
    # Convert message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    message_length = len(binary_message)
    
    # Embed message in frame
    frame_height, frame_width, _ = frame.shape
    idx = 0
    for y in range(frame_height):
        for x in range(frame_width):
            if idx < message_length:
                # Convert pixel value to binary
                pixel_binary = format(frame[y, x], '08b')
                # Replace the least significant bit of each color channel with the message bit
                new_pixel_binary = pixel_binary[:-1] + binary_message[idx]
                # Convert back to integer
                frame[y, x] = int(new_pixel_binary, 2)
                idx += 1
            else:
                break
        if idx >= message_length:
            break
    return frame

# Function to extract message from a video frame
def extract_message(frame, message_length):
    extracted_message = ''
    frame_height, frame_width, _ = frame.shape
    idx = 0
    for y in range(frame_height):
        for x in range(frame_width):
            pixel_binary = format(frame[y, x], '08b')
            # Extract the least significant bit of each color channel
            extracted_message += pixel_binary[-1]
            idx += 1
            if idx == message_length:
                return ''.join(chr(int(extracted_message[i:i+8], 2)) for i in range(0, len(extracted_message), 8))

# Function to hide message in video
def hide_message_in_video(video_path, message, output_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Embed message in each frame
    for _ in range(frame_count):
        ret, frame = cap.read()
        if ret:
            frame_with_message = embed_message(frame, message)
            out.write(frame_with_message)
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to extract message from video
def extract_message_from_video(video_path, message_length):
    cap = cv2.VideoCapture(video_path)
    message = ''
    for _ in range(message_length):
        ret, frame = cap.read()
        if ret:
            extracted_message = extract_message(frame, message_length)
            message += extracted_message
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    return message

# Example usage:
video_path = 'input_video.mp4'
output_video_path = 'output_video_hidden_message.mp4'
message_to_hide = "This is a hidden message!"

# Hide message in video
hide_message_in_video(video_path, message_to_hide, output_video_path)

# Extract message from video
extracted_message = extract_message_from_video(output_video_path, len(message_to_hide))
print("Extracted message:", extracted_message)


Extracted message: 
